In [1]:
pip install --user --force-reinstall matplotlib

  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/7b/e5/83fcd7e9db036c179e0352bfcd20f81d728197a16f883e7b90307a88e65e/seaborn-0.13.0-py3-none-any.whl.metadata
  Using cached seaborn-0.13.0-py3-none-any.whl.metadata (5.3 kB)
  Obtaining dependency information for numpy!=1.24.0,>=1.20 from https://files.pythonhosted.org/packages/2d/ed/022fc4106f6d97e41e156201274138e0369b27dbfc8c206034f24ebd97d9/numpy-1.26.1-cp39-cp39-win_amd64.whl.metadata
  Using cached numpy-1.26.1-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for pandas>=1.2 from https://files.pythonhosted.org/packages/f2/8c/35364a11b3e25f8e29a35420b0d18f65ec4f9d6d38e86a62d16ef998923c/pandas-2.1.1-cp39-cp39-win_amd64.whl.metadata
  Using cached pandas-2.1.1-cp39-cp39-win_amd64.whl.metadata (18 kB)
  Obtaining dependency information for matplotlib!=3.6.1,>=3.3 from https://files.pythonhosted.org/packages/01/50/0d8d8f044e2a0d8151e9ed59fe50924e9e697ba43a8b12d5ff9b45ad

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\asche\\anaconda3\\envs\\Test\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSans-Oblique.ttf'
Consider using the `--user` option or check the permissions.



In [2]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

AttributeError: module 'matplotlib._api' has no attribute 'MatplotlibDeprecationWarning'

# load dataset 1

In [ ]:
df = pd.read_csv('1_downloaded_data.csv', low_memory=False)

# 2 Action_taken and applicant_sex distribution

We want the distribution of the target variable: action_taken.

- 1 - Loan originated
- 2 - Application approved but not accepted
- 3 - Application denied
- 4 - Application withdrawn by applicant
- 5 - File closed for incompleteness
- 6 - Purchased loan
- 7 - Preapproval request denied
- 8 - Preapproval request approved but not accepted

An we also want the distibution of the applicant_sex variable.

- 1 - Male
- 2 - Female
- 3 - Information not provided by applicant in mail, internet, or telephone application
- 4 - Not applicable
- 6 - Applicant selected both male and female

In [ ]:
df['action_taken'].value_counts()

# Remove anything except male, female and mixed gender.

In [ ]:
# Remove rows where applicant_sex is 3 or 4, and also remove null values
df_filtered = df[~df['applicant_sex'].isin([3, 4])].dropna(subset=['applicant_sex'])

# Check the new value counts
print(df_filtered['applicant_sex'].value_counts())

## Plotting Histograms

In [ ]:
# Create a histogram on a logarithmic scale for the y-axis
plt.hist(df_filtered['action_taken'], bins=30, edgecolor='black', log=False)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Values of action_taken')

# Show the plot
plt.show()

In [ ]:
# Create a histogram on a logarithmic scale for the y-axis
plt.hist(df_filtered['applicant_sex'], bins=30, edgecolor='black', log=False)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Values of derived_sex')

# Show the plot
plt.show()

### Shapiro-Wilk Test for Normality
assess if the combined data is normally distributed.

In [ ]:
# Extract data for 'action_taken' and 'applicant_sex'
data_combined1 = df_filtered[['action_taken', 'applicant_sex']]

In [ ]:
data_combined1.values

In [ ]:
# Perform Shapiro-Wilk test
stat_combined1, p_value_combined1 = shapiro(data_combined1)

# Print the results
print('Shapiro-Wilk Test for Combined Data:')
print('  Statistic:', stat_combined1)
print('  p-value:', p_value_combined1)
print('  Is the data normally distributed?', p_value_combined1 > 0.05)


### Test for normal, exponential, poisson, gamma distribution

In [ ]:
data = data_combined1[['action_taken', 'applicant_sex']].values

# Flatten the data into a 1D array
flattened_data = data.flatten()
flattened_data = flattened_data[flattened_data >= 0]  # Filter out negative or invalid values

# Fit the data to different distributions
params_norm = norm.fit(flattened_data)
params_expon = expon.fit(flattened_data)

# Generate x values for the PDFs
x = np.linspace(min(flattened_data), max(flattened_data), 1000)

# Fit the Poisson distribution
mu_poisson = np.mean(flattened_data)
pdf_poisson = poisson.pmf(x.astype(int), mu_poisson)

# Fit a truncated gamma distribution (truncated at 0 to avoid negative values)
a, loc, scale = gamma.fit(flattened_data, floc=0)
pdf_trunc_gamma = gamma.pdf(x, a, loc=0, scale=scale)

# Generate x values for the PDFs
x = np.linspace(min(flattened_data), max(flattened_data), 1000)
pdf_norm = norm.pdf(x, *params_norm)
pdf_expon = expon.pdf(x, *params_expon)

# Plot the data and fits
plt.hist(flattened_data, bins=30, density=True, alpha=0.7, label="Data histogram")
plt.plot(x, pdf_norm, label="Fitted normal distribution")
plt.plot(x, pdf_expon, label="Fitted exponential distribution")
plt.plot(x, pdf_poisson, label="Fitted Poisson distribution")
plt.plot(x, pdf_trunc_gamma, label="Fitted Truncated Gamma distribution")

plt.xlabel("The Data", fontsize=12)  # Adjust the fontsize here
plt.ylabel("Probability Density", fontsize=12)  # Adjust the fontsize here
plt.legend()
plt.legend(fontsize=10)  # Adjust the fontsize for the legend here
plt.xticks(fontsize=10)  # Adjust the fontsize for the x-axis ticks here
plt.yticks(fontsize=10)  # Adjust the fontsize for the y-axis ticks here

plt.show()

## Plotting Boxplots

In [ ]:
# Create a list of categories (unique values in 'derived_sex')
categories = df_filtered['applicant_sex'].unique()
categories

# Initialize a list to store the data for each category
data_per_category = []


# Iterate over each category and extract the 'action_taken' values
for category in categories:
    data_per_category.append(df_filtered[df_filtered['applicant_sex'] == category]['action_taken'].values)

data_per_category 

# Create a boxplot
# plt.boxplot(data_per_category, labels=categories)

# Add labels and title
# plt.xlabel('applicant_sex')
# plt.ylabel('Action Taken')
# plt.title('Boxplot of Action Taken vs applicant_sex')

# Show the plot
# plt.show()

# Violin Plot

In [ ]:
# Create a violin plot
sns.violinplot(x='applicant_sex', y='action_taken', data=df_filtered)

# Add labels and title
plt.xlabel('Applicant Sex')
plt.ylabel('Action Taken')
plt.title('Violin Plot of Action Taken vs Applicant Sex')

# Show the plot
plt.show()

#  the white dot at the center of each "violin" represents the median of the data within that category. 
#  The thicker part of the violin (the "body") represents the interquartile range (IQR), which contains the middle 50% of the data.
#  The thinner sections extending from the body (the "whiskers") indicate the data's minimum and maximum values within 1.5 times the IQR.

##  Barplot

In [ ]:
pip install --upgrade seaborn matplotlib

In [ ]:
# Filter out the warnings related to is_categorical_dtype and use_inf_as_na
warnings.filterwarnings("ignore", message="is_categorical_dtype is deprecated")
warnings.filterwarnings("ignore", message="use_inf_as_na option is deprecated")

first_dimension = "action_taken"
horizontal_label = "x label"
second_dimension = "applicant_sex"

sns.histplot(binwidth=1,
            x=first_dimension,
            hue=second_dimension,
            data=df_filtered,
            stat="count",
            multiple="dodge",
            palette=['lightgreen', 'pink', 'yellow', 'orange', 'grey'],
            )  # Adjust the number of bins as needed.


In [ ]:
df_test = df_filtered.copy()

# Group by 'action_taken' and 'applicant_sex' and count the occurrences
grouped_data = df_test.groupby(['action_taken', 'applicant_sex']).size().unstack().fillna(0)

# Calculate the total count of each 'action_taken' category
total_counts_action_taken = grouped_data.sum(axis=1)

# Calculate the percentages for each combination
percentage_data = grouped_data.divide(total_counts_action_taken, axis=0) * 100

# Print the percentage data
print("Percentage of each 'action_taken' category for each 'applicant_sex':")
print(percentage_data)

In [ ]:

# Filter data for 'action_taken' categories 1 and 3
filtered_data = df_test[df_test['action_taken'].isin([1, 3])]

# Group by 'action_taken' and 'applicant_sex' and count the occurrences
grouped_data = filtered_data.groupby(['action_taken', 'applicant_sex']).size().unstack().fillna(0)

# Calculate the total count of each 'action_taken' category
total_counts_action_taken = grouped_data.sum(axis=1)

# Calculate the percentages for each combination
percentage_data = grouped_data.divide(total_counts_action_taken, axis=0) * 100

# Plot the percentages for 'action_taken' categories 1 and 3
percentage_data.loc[[1, 3]].T.plot(kind='bar', stacked=True)

# Add labels and title
plt.xlabel('Applicant Sex')
plt.ylabel('Percentage')
plt.title('Percentage of Action Taken Categories 1 and 3 by Applicant Sex')

# Show the plot
plt.show()

# Conclusion

The exponential distribution is often used to model the time until an event occurs in a Poisson process, which could be related to the time until a loan is approved or some other event.


- Data: Sex and loan approval (action taken category).

- Distribution: Exponential distribution, modeling the time until loan approval.

- Interpretation: The exponential distribution suggests that the time (or other relevant measure) until a loan is approved may follow this distribution. The rate parameter of the exponential distribution (often denoted as λ, which is the reciprocal of the mean) provides an estimate of how frequently loans are approved.

In [ ]:
import numpy as np

# Filter data for applicant_sex category 1 and action_taken category 1
data_applicant_sex_1_action_taken_1 = df_test[df_test['applicant_sex'] == 1]['action_taken']

# Calculate the mean of the filtered data
mean_of_data_applicant_sex_1_action_taken_1 = np.mean(data_applicant_sex_1_action_taken_1)

# Calculate the rate parameter (lambda) for applicant_sex category 1 and action_taken category 1
rate_parameter_applicant_sex_1_action_taken_1 = 1 / mean_of_data_applicant_sex_1_action_taken_1

print("Rate parameter (lambda) for the exponential distribution, applicant_sex category 1 and action_taken:", rate_parameter_applicant_sex_1_action_taken_1)

In [ ]:
import numpy as np

# Filter data for applicant_sex category 1 and action_taken category 1
data_applicant_sex_2_action_taken_1 = df_test[df_test['applicant_sex'] == 2]['action_taken']

# Calculate the mean of the filtered data
mean_of_data_applicant_sex_2_action_taken_1 = np.mean(data_applicant_sex_2_action_taken_1)

# Calculate the rate parameter (lambda) for applicant_sex category 1 and action_taken category 1
rate_parameter_applicant_sex_2_action_taken_1 = 1 / mean_of_data_applicant_sex_2_action_taken_1

print("Rate parameter (lambda) for the exponential distribution, applicant_sex category 2 and action_taken:", rate_parameter_applicant_sex_2_action_taken_1)


In [ ]:
df_test.to_csv('2_distribution_applicant_sex_and_action_taken.csv', index=False)